**<center><font size=5>EEG Data Analysis and Machine Learning(Clustering)</font></center>**
***<center>Alcoholic vs Control Groups</center>***
***
**Autor**: Iván García Alvarez

**Fecha**: 23 Marzo del 2019

**[GitHub Repository](https://github.com/navi931/EEG-alcohol)**

#### Table of Contents
- <a href='#intro'>1. Project Overview</a> 
- <a href='#env'>2. Setting up the Environment</a>
- <a href='#import'>2.1. Data Import</a>
- <a href='#Preprocessing'>2.2 Preprocessing</a>
- <a href='#Training'>3 Training</a> 
- <a href='#Testing'>4 Testing</a> 
- <a href='#Conclusiones'>5 Conclusiones</a>

# <a id='intro'>1.Project Overview</a>

Este proyecto ha sido publicado en Kaggle [EEG-Alcohol](https://www.kaggle.com/ruslankl/eeg-data-analysis) como un reto para implementar Machine Learning, así que hemos tomado el reto y en este proyecto implementaremos Support Vector Machines.Al igual que el estudio de kaggle he escogido el [EEG-Alcohol](https://www.kaggle.com/nnair25/Alcoholics) dataset, el cual [EEG (Electroencephalography)](https://en.wikipedia.org/wiki/Electroencephalography) que son los datos de dos grupos - Alcoholic and Control Group. 
![](https://i.imgur.com/ZrmxJRu.jpg)
La cantidad de sujetos en cada grupo es 8. Los 64 electrodos se colocaron en el cuero cabelludo del sujeto para medir la actividad eléctrica del cerebro. Los valores de respuesta se muestrearon a 256 Hz (época de 3.9 ms) durante 1 segundo. Cada sujeto fue expuesto a un solo estímulo (S1) o a dos estímulos (S1 y S2) que eran imágenes de objetos elegidos de entre [1980 Snodgrass and Vanderwart picture set](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.294.1979&rep=rep1&type=pdf). Cuando se mostraron dos estímulos, se presentaron en una condición emparejada donde S1 era idéntico a S2 o en una condición no emparejada donde S1 difería de S2.

El propósito del algoritmo será detectar diferencias entre los valores de respuesta para diferentes estímulos entre el control y el grupo alcohólico.

# <a id='env'>2. Setting up the Environment</a>
## <a id='import'>2.1. Data Import</a>

In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, model_selection,metrics
from sklearn.cluster import KMeans
from tqdm import tqdm_notebook as tqdm
import os
import warnings
import random
warnings.filterwarnings("ignore")
%matplotlib inline 

# <a id='Preprocessing'>Preprocessing</a>
Lo primero que haremos será hacer el preprcesamiento de los datos, tomaremos en cuenta las siguientes recomendaciones para esto:
* No nos interesa un "id" del sujeto, pues puede ocasionar un overfitting

* Nos interesa mucho cuales sensores son los sensores importantes, para esto tomaremos las conclusiones dadas por el estudio previo que son 'FPZ', 'FP2', 'AF3', 'AFZ', 'AF4', 'F5', 'F3', 'F1', 'FZ', 'FC5', 'FC3', 'FCZ', 'T7', 'CZ', 'C4', 'C6','TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP8', 'P5', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO4', 'O1'

* Analizando los datos nos dimos cuenta que la posicion del sensor y el canal, es lo mismo entonces por motivos de procesamiento hemos decidir eliminar el campo chanel.

* Para un mejor rendimiento del algoritmo se ha decidido dejar todo en valores númericos

Enseguida procederemos a obtener los datos y cargarlos a python

In [2]:
eeg_data_S1Obj = pd.read_csv("eeg_datasets_split/eeg_data_S1Obj.csv",encoding='latin-1')
eeg_data_S2Match = pd.read_csv("eeg_datasets_split/eeg_data_S2Match.csv",encoding='latin-1')
eeg_data_S2NoMatch = pd.read_csv("eeg_datasets_split/eeg_data_S2NoMatch.csv",encoding='latin-1')
eeg_data = [eeg_data_S1Obj,eeg_data_S2Match,eeg_data_S2NoMatch]
eeg_data_S1Obj.head(n=10)

,Target,d0,d1,d2,d3,d4,d5,d6,d7,d8,...,d8438,d8439,d8440,d8441,d8442,d8443,d8444,d8445,d8446,d8447
0,c,0.081,0.824,4.364,4.842,3.703,1.200,4.323,5.117,4.201,...,-8.586,-9.206,-10.030,-12.726,-7.284,-5.259,-5.208,2.218,-7.324,-7.853
1,a,-0.214,-4.639,-5.066,-5.910,-5.188,-2.787,-5.147,-6.856,-6.124,...,-3.998,-6.571,-3.988,13.234,-1.119,-1.740,-7.517,-0.753,34.576,-7.111
2,c,-2.757,-0.590,-1.149,0.397,0.285,-4.720,-2.309,0.387,0.763,...,-17.192,-24.638,-13.855,-18.250,-12.421,-11.688,-10.142,-5.514,-7.314,-10.579
3,c,0.051,-2.838,2.523,0.987,-1.648,3.042,3.550,1.984,-0.153,...,0.407,6.022,5.219,2.736,6.276,6.683,6.165,2.024,1.373,3.184
4,c,4.883,7.945,3.957,4.842,4.771,3.245,2.686,2.553,2.472,...,-20.182,-23.448,-2.604,-15.940,-24.424,-25.645,-25.981,-20.579,-20.691,-24.506
5,c,4.822,6.521,0.366,1.119,2.411,-1.068,-0.804,-0.437,-0.173,...,-3.754,-5.941,-1.414,-2.116,0.051,-4.852,-2.574,9.288,-3.560,-8.006
6,a,-3.540,-8.840,-2.340,-3.448,-5.422,0.275,-1.343,-2.950,-2.309,...,-1.892,-2.706,-2.950,-13.763,-0.621,-0.478,0.814,5.636,-2.696,-3.092
7,c,-4.730,-4.547,-4.140,-2.635,-2.543,-3.042,-1.302,-1.373,-0.610,...,-2.502,-3.855,-3.520,-4.761,-7.701,-4.120,-3.276,-4.517,-3.194,-2.767
8,c,-0.193,-0.783,2.401,4.751,4.781,-4.445,-1.495,2.869,6.419,...,2.858,7.741,8.881,7.334,4.791,2.370,4.801,16.093,-0.264,7.477
9,a,-5.554,-7.955,-4.761,-6.104,-7.192,-2.513,-3.581,-6.429,-3.998,...,-12.950,-3.438,-6.093,-13.540,-8.769,-15.544,-11.149,-11.739,10.193,-15.096


Enseguida lo que haremos sera clasificar los alcoholicos 'a' como 1 y grupo de control como 0 (campo a clasificar)
además que solo usaremos los campos "sensor_position","sample_num","sensor_value","matching_condition".

In [3]:
X = [0,1,2]
Y = [0,1,2]
index = 0
for data in tqdm(eeg_data):
    data["Target"]= data["Target"].map({'a':1,'c':0})
    Y[index] = np.array(data["Target"])
    data = np.array(data)
    X[index] = np.delete((np.array(data)), 0, 1)
    index = index + 1;

X_train_s1, X_test_s1, y_train_s1, y_test_s1 = model_selection.train_test_split(X[0],Y[0], test_size=0.25, random_state=42)
X_train_s2_match, X_test_s2_match, y_train_s2_match, y_test_s2_match = model_selection.train_test_split(X[1],Y[1],test_size=0.25, random_state=42)
X_train_s2_no_match, X_test_s2_no_match, y_train_s2_no_match, y_test_s2_no_match = model_selection.train_test_split(X[2],Y[2],test_size=0.25, random_state=42)


In [4]:
for prueba in tqdm(X_train_s1):
    i = 0
    while(i<len(prueba)):
            if(prueba[i] < 0):
                prueba[i] = prueba[i]
            i = i + 1
print(X_train_s1)


[[  4.364  10.345   6.51  ...  27.629   4.201   9.125]
 [ -3.55   -4.191  -3.52  ...  15.289   4.7    11.088]
 [ -3.937  -4.476  -6.083 ... -34.658  -0.997  -3.204]
 ...
 [  2.126   1.292  -1.007 ...  -7.05   -8.789  -4.344]
 [  4.832   4.476   7.121 ... -13.163 -10.895  -7.385]
 [  8.301   8.799   6.541 ...  10.854   1.973  -2.604]]


# <a id='Training'>Training</a>
Teniendo los datasets de train y de prueba procederemos a entrenar el modelo.

In [5]:
cluster1 = KMeans(n_clusters=2, random_state=0)
cluster1.fit(X_train_s1, y_train_s1)

cluster2 = KMeans(n_clusters=2, random_state=0)
cluster2.fit(X_train_s2_match, y_train_s2_match)

cluster3 = KMeans(n_clusters=2, random_state=0)
cluster3.fit(X_train_s2_no_match, y_train_s2_no_match)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

# <a id='Testing'>Testing</a>
Ya entrenado el modelo procederemos a medir diferentes mediciones 
como el accuracy,precision,recall y la matriz de confusion

In [6]:
print("Modelo de Cluster1")
print("")
m_confusion_test_cluster1 = metrics.confusion_matrix(y_test_s1, cluster1.predict(X_test_s1))
print(pd.DataFrame(data = m_confusion_test_cluster1, columns = ['Predicted no alcoholic', 'Predicted alcoholic'],
            index = ['Actual no alcoholic', 'Actual alcoholic']))

print("score",cluster1.score(X_test_s1, y_test_s1))
print("recall",metrics.recall_score(y_test_s1, cluster1.predict(X_test_s1)))
print("precision",metrics.precision_score(y_test_s1, cluster1.predict(X_test_s1)))
print("")

print("Modelo de Cluster2")
print("")
m_confusion_test_cluster2 = metrics.confusion_matrix(y_test_s2_match, cluster1.predict(X_test_s2_match))
print(pd.DataFrame(data = m_confusion_test_cluster2, columns = ['Predicted no alcoholic', 'Predicted alcoholic'],
            index = ['Actual no alcoholic', 'Actual alcoholic']))

print("score",cluster2.score(X_test_s2_match, y_test_s2_match))
print("recall",metrics.recall_score(y_test_s2_match, cluster2.predict(X_test_s2_match)))
print("precision",metrics.precision_score(y_test_s2_match, cluster2.predict(X_test_s2_match)))
print("")

print("Modelo de Cluster3")
print("")
m_confusion_test_cluster3 = metrics.confusion_matrix(y_test_s2_no_match, cluster3.predict(X_test_s2_no_match))
print(pd.DataFrame(data = m_confusion_test_cluster3, columns = ['Predicted no alcoholic', 'Predicted alcoholic'],
            index = ['Actual no alcoholic', 'Actual alcoholic']))

print("score",cluster3.score(X_test_s2_no_match, y_test_s2_no_match))
print("recall",metrics.recall_score(y_test_s2_no_match, cluster3.predict(X_test_s2_no_match)))
print("precision",metrics.precision_score(y_test_s2_no_match, cluster3.predict(X_test_s2_no_match)))
print("")


Modelo de Cluster1

                     Predicted no alcoholic  Predicted alcoholic
Actual no alcoholic                      23                   16
Actual alcoholic                         14                   24
score -35091595.70514284
recall 0.631578947368421
precision 0.6

Modelo de Cluster2

                     Predicted no alcoholic  Predicted alcoholic
Actual no alcoholic                      31                    7
Actual alcoholic                         30                   10
score -50244172.87909577
recall 0.575
precision 0.6571428571428571

Modelo de Cluster3

                     Predicted no alcoholic  Predicted alcoholic
Actual no alcoholic                      16                   20
Actual alcoholic                         16                   24
score -39484143.91468835
recall 0.6
precision 0.5454545454545454



# <a id='Pruebas personales'>Pruebas reales</a>
Teniendo el modelo entrenado veremos como clasifica ciertas personas random.
Tomaremos a 3 personas, una para cada clasificador, y veremos como las clasifica.

In [7]:
modelos = [cluster1,cluster2,cluster3]
aleatorio = random.randint(0, 50)
test = [0,0,0]
diagnosticos = [0,0,0]
test[0] = X_test_s1
test[1] = X_test_s2_match
test[2] = X_test_s2_no_match
diagnosticos[0] = y_test_s1[aleatorio]
diagnosticos[1] = y_test_s2_match[aleatorio]
diagnosticos[2] = y_test_s2_no_match[aleatorio]
# 'alcoholico' if (modelo.predict(test[index])[aleatorio]==1) else 'no alcoholico'
# 'alcoholico' if (diagnosticos[index][aleatorio]==1) else 'no alcoholico'
index = 0

predicts = [0,0,0]

for modelo in modelos:
    predicts[index] = modelo.predict(test[index])[aleatorio]
    index = index + 1
    
index = 0
for modelo in modelos:
    print('El algoritmo dijo que','si es alcoholico' if (predicts[index]==1) else 'no es alcoholico',
          'y en realidad','si es alcoholico' if (diagnosticos[index]==1) else 'no es alcoholico')
    index = index + 1

El algoritmo dijo que si es alcoholico y en realidad no es alcoholico
El algoritmo dijo que no es alcoholico y en realidad si es alcoholico
El algoritmo dijo que no es alcoholico y en realidad si es alcoholico


# <a id='Conclusiones'>Conclusiones</a>
Si utiizamos estos tres clasificadores para una conclusion final sería bastante efectivo, pues haciendole las tres pruebas a un individuo, las clasificamos y sacamos la moda de estas, podemos predecir muy bien si es alcoholico o no.